## Bootstrap

In [1]:
import numpy as np 
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
%matplotlib inline

Дана выборка из генеральной совокупности и необходимо оценить выбоочное среднее $\theta$. Сформируем $N$ подвыборок, которые будут формироваться случайным выбором с возвращением, выбранные элементы исходной выборки возвращаются в выборку и далее могут быть использованы снова.

Формально на каждом шаге мы выбираем жлемент исходной выборки с вероятностью $\frac{1}{n}$, поэтому из генеральной совокупности будут *не выбраны*:
$$\lim\limits_{n\ \to \infty} \left(1-\frac{1}{n}\right)^n = e^{-1} \approx 0.36$$

Для построения доверительного интервала:

- построим $m=1000$ подвыборок с возвращением
- посчитаем для каждой выборке:
    - выборочное среднее: $$\bar{X} = \frac{1}{n}\sum_{i=1}^n x_i$$
    - выборочное стандартное отклонение (standard deviation (SD)): $$S_0=\sqrt{\frac{n}{n-1}S^2}=\sqrt{\frac{1}{n-1}\sum_{i=1}^n\left(x_i-\overline{x}\right)^2}$$
    - посчитаем Standard Error of Mean - $$SEM = \frac{S_0}{\sqrt{n}}$$

Результатом будет являться:
$$E\xi : \overline{X} \mp SEM$$

## Functions

In [2]:
def sample_bootstrap(data, n_samples):
    '''
    функция реализует возвращение подвыборок из генеральной совокупности
    
    ----
    
    data - исходные данные
    n_samples - количество разбиений 
    
    output: индексы элементов
    '''
    n = len(data)
    idx = np.random.randint(low = 0, high = n, size = (n_samples, n))
    samples = data[idx]
    return samples

def stat_intervals(stat,alpha=0.05):
    '''
    функция возвращает доверительный интервал вторым способом
    
    alpha - доверительный уровень
    
    stat - массив статистик (средних в нашем случае)
    
    '''
    bound = np.percentile(sorted(stat),[100*alpha/2.,100*(1-alpha/2.)])
    
    return bound

### Preprocessing

In [3]:
directory = 'data/*.txt' 
files = glob.glob(directory)

for i, file in enumerate(files):
    exec(f"x_{i} = pd.read_csv(files[i], header=None, sep = '\t',names = {[str(file)]}, decimal=',') ")

## Results

In [9]:
# np.random.seed(0)
alpha = 0.05
result = pd.DataFrame(columns = [r'mean: $\overline{X}$', r'SEM: $\frac{S_0}{\sqrt{n}}$', r'E$\xi$: mean $\mp$ sem'])
a = np.array([])

for i,x in enumerate([x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8]):
    
    x_0_mean_score = np.array(list(map(np.mean, sample_bootstrap(x.values,100000)))) #bootstrap выборки
    x_0_standard_deviation_score = np.array(list(map(np.std, sample_bootstrap(x.values,100000)))) #bootstrap выборки
    
    mean_bootstrap = np.mean(x_0_mean_score) #среднее по boostratrp средним
    standard_deviation_bootstrap = np.mean(x_0_standard_deviation_score) #среднее по boostratrp средним
    
#     left = mean_bootstrap - standard_deviation_bootstrap/np.sqrt(x.values)  #левая граница 1-ого способа
#     right = mean_bootstrap +  standard_deviation_bootstrap/np.sqrt(x.values) #правая граница 2-ого способа
    standatd_error_of_mean = standard_deviation_bootstrap/np.sqrt(len(x.values))
    results = f'{mean_bootstrap:.4} $\mp$ {standatd_error_of_mean:.4}'
    
    result.loc[x.columns[0]]=np.array([ mean_bootstrap,  standatd_error_of_mean, results])

In [10]:
result[result.columns[:-1]]=result[result.columns[:-1]].astype(float).round(3)
result

,mean: $\overline{X}$,SEM: $\frac{S_0}{\sqrt{n}}$,E$\xi$: mean $\mp$ sem
data\10_uM_STL.txt,192.005,41.794,192.0 $\mp$ 41.79
data\200_uM_ITP.txt,122.864,37.204,122.9 $\mp$ 37.2
data\200_uM_ITP_10_uM_STL.txt,294.552,71.722,294.6 $\mp$ 71.72
data\400_uM_NTP.txt,61.550,16.192,61.55 $\mp$ 16.19
data\400_uM_NTP_10_uM_STL.txt,346.320,105.344,346.3 $\mp$ 105.3
data\5_uM_GreA_10_uM_STL.txt,216.937,46.919,216.9 $\mp$ 46.92
data\5_uM_GreA_D41A.txt,273.570,97.670,273.6 $\mp$ 97.67
data\5_uM_GreA_D41A_10_uM_STL.txt,311.748,46.395,311.7 $\mp$ 46.4
data\5_uM_GreB_10_uM_STL.txt,388.177,56.485,388.2 $\mp$ 56.49
